In [1]:
class Polynomial:
    def __init__(self, coefficients, p=2):
        """
        Инициализация полинома с коэффициентами в поле GF(p).
        coefficients: список коэффициентов, начиная со старшего разряда.
        p: простое число, определяющее поле GF(p).
        """
        self.coefficients = [c % p for c in coefficients]
        self.p = p

    def __add__(self, other):
        """Сложение полиномов в поле GF(p)."""
        max_len = max(len(self.coefficients), len(other.coefficients))
        padded_self = [0] * (max_len - len(self.coefficients)) + self.coefficients
        padded_other = [0] * (max_len - len(other.coefficients)) + other.coefficients
        result = [(a + b) % self.p for a, b in zip(padded_self, padded_other)]
        return Polynomial(result, self.p)

    def __mul__(self, other):
        """Умножение полиномов в поле GF(p)."""
        result = [0] * (len(self.coefficients) + len(other.coefficients) - 1)
        for i, a in enumerate(self.coefficients):
            for j, b in enumerate(other.coefficients):
                result[i + j] = (result[i + j] + a * b) % self.p
        return Polynomial(result, self.p)

    def __mod__(self, other):
        """Деление полиномов в поле GF(p) (возвращает остаток)."""
        dividend = self.coefficients[:]
        divisor = other.coefficients[:]
        while len(dividend) >= len(divisor):
            shift = len(dividend) - len(divisor)
            quotient_term = (dividend[0] * pow(divisor[0], -1, self.p)) % self.p
            for i in range(len(divisor)):
                dividend[i + shift] = (dividend[i + shift] - quotient_term * divisor[i]) % self.p
            # Удалить ведущие нули
            while dividend and dividend[0] == 0:
                dividend.pop(0)
        return Polynomial(dividend, self.p)

    def __str__(self):
        """Строковое представление полинома."""
        terms = []
        for i, coeff in enumerate(self.coefficients):
            if coeff != 0:
                power = len(self.coefficients) - i - 1
                term = f"{coeff}" if power == 0 else f"{coeff}x^{power}"
                terms.append(term)
        return " + ".join(terms) if terms else "0"


class ShiftRegister:
    def __init__(self, n, initial_state, feedback_polynomial, p=2):
        """
        Инициализация регистра сдвига.
        n: количество регистров.
        initial_state: начальное состояние регистров (список длины n).
        feedback_polynomial: полином обратной связи (список коэффициентов).
        p: простое число, определяющее поле GF(p).
        """
        self.n = n
        self.state = initial_state[:]
        self.feedback_polynomial = Polynomial(feedback_polynomial, p)
        self.p = p

    def shift(self):
        """Выполнить один такт сдвига."""
        # Вычислить новый бит на основе полинома обратной связи
        feedback_bit = sum(
            coeff * self.state[-i - 1] for i, coeff in enumerate(self.feedback_polynomial.coefficients)
        ) % self.p
        # Сдвинуть регистр
        self.state = [feedback_bit] + self.state[:-1]

    def multiply(self, polynomial):
        """Умножение текущего состояния на полином."""
        current_state_poly = Polynomial(self.state, self.p)
        result = current_state_poly * polynomial
        return result

    def divide(self, polynomial):
        """Деление текущего состояния на полином (возвращает остаток)."""
        current_state_poly = Polynomial(self.state, self.p)
        remainder = current_state_poly % polynomial
        return remainder

    def __str__(self):
        """Строковое представление состояния регистра."""
        return f"State: {self.state}"


# Параметры
n = 4  # Количество регистров
p = 2  # Поле GF(2)
initial_state = [1, 0, 1, 1]  # Начальное состояние
feedback_polynomial = [1, 0, 0, 1]  # Полином обратной связи x^3 + 1

# Создание регистра сдвига
sr = ShiftRegister(n, initial_state, feedback_polynomial, p)

print("Исходное состояние:")
print(sr)

# Умножение на полином
poly_to_multiply = Polynomial([1, 1], p)  # x + 1
result_multiply = sr.multiply(poly_to_multiply)
print("\nРезультат умножения на полином:")
print(result_multiply)

# Деление на полином
poly_to_divide = Polynomial([1, 0, 1], p)  # x^2 + 1
result_divide = sr.divide(poly_to_divide)
print("\nОстаток от деления на полином:")
print(result_divide)

# Выполнение одного такта сдвига
sr.shift()
print("\nСостояние после одного такта сдвига:")
print(sr)

Исходное состояние:
State: [1, 0, 1, 1]

Результат умножения на полином:
1x^4 + 1x^3 + 1x^2 + 1


KeyboardInterrupt: 